<a href="https://colab.research.google.com/github/16pynkn/MSC_DPDM_2025/blob/main/Frequent_Patterns_(Association_Rules).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Dataset_dpdm25'

In [ ]:
data = pd.read_csv(os.path.join(path,'reduced_marketbasket.csv'),encoding='latin-1') #encoding= 'latin-1' bc. มีภาษาที่เป็นจุด ๆ บนหัวทำให้ข้อมมูลเละ

In [ ]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/01/2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/01/2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/01/2010 08:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/09/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/09/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/09/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/09/2011 12:50,4.15,12680.0,France


In [ ]:
data.isnull().any()

,0
InvoiceNo,False
StockCode,False
Description,True
Quantity,False
InvoiceDate,False
UnitPrice,False
CustomerID,True
Country,False


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## การเตรียมข้อมูลสำหรับ Apriori Algorithm

ข้อมูลที่ใช้มักจะอยู่ในรูปแบบของธุรกรรม (Transactions) โดยแต่ละธุรกรรมจะประกอบไปด้วยชุดของรายการ (Item)

InvoiceNo = รหัสใบเสร็จ, StockCode = รหัสสินค้า, Description = คำอธิบายรายการสินค้า, Quantity = จำนวนชิ้น, InvoiceDate = วันที่ซื้อ, UnitPrice = ราคาต่อหน่วย, CustomerID = รหัสสมาชิกลูกค้า, Country = ประเทศที่ตั้ง กรณีมีหลายสาขา

In [ ]:
transactions = data.groupby('InvoiceNo')['StockCode'].apply(list) #จัดกลุ่มแล้วสร้างเป็น list #รวม ให้สินค้ารหัสใบเสร็จเดียวกันมาอยู่ด้วยกัน # จัดกลุ่มข้อมูล .apply(func  ที่จะทำ)

In [ ]:
transactions

,StockCode
InvoiceNo,
536365,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
536366,"[22633, 22632]"
536367,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
536368,"[22960, 22913, 22912, 22914]"
536369,[21756]
...,...
C581484,[23843]
C581490,"[22178, 23144]"
C581499,[M]


In [ ]:
len(data['Country'].unique())

38

In [ ]:
set(data['Country']) #  เหมือนกับ .unique() สามารถระบุได้ไหมว่าลูกค้าประเทศไหนซื้อสินค้าอะไร

{'Australia',
 'Austria',
 'Bahrain',
 'Belgium',
 'Brazil',
 'Canada',
 'Channel Islands',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'EIRE',
 'European Community',
 'Finland',
 'France',
 'Germany',
 'Greece',
 'Hong Kong',
 'Iceland',
 'Israel',
 'Italy',
 'Japan',
 'Lebanon',
 'Lithuania',
 'Malta',
 'Netherlands',
 'Norway',
 'Poland',
 'Portugal',
 'RSA',
 'Saudi Arabia',
 'Singapore',
 'Spain',
 'Sweden',
 'Switzerland',
 'USA',
 'United Arab Emirates',
 'United Kingdom',
 'Unspecified'}

## Association Rules

In [ ]:
!pip install apyori

  Preparing metadata (setup.py) ... done
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5954 sha256=4251536648c3ab982fcb6e9f395037fc90d83991b82dd71e3b159773d346d11d
  Stored in directory: /root/.cache/pip/wheels/7f/49/e3/42c73b19a264de37129fadaa0c52f26cf50e87de08fb9804af
Successfully built apyori


In [ ]:
from apyori import apriori

In [ ]:
list(apriori(transactions,min_support = 0.5)) #ไม่มีสินค้าใดๆที่ถูกซื้อ เกิน 50% จากtransactions

[]

In [ ]:
list(apriori(transactions,min_support = 0.1)) #ไม่มีสินค้าใดๆที่ถูกซื้อ เกิน 10% จากtransactions

[]

In [ ]:
frequentItemsets = list(apriori(transactions,min_support = 0.065)) #output 4 lists มี itemset 4 ตัวที่ถูกซื้อเกิน 6.5%
frequentItemsets

[RelationRecord(items=frozenset({'22423'}), support=0.08386100386100385, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22423'}), confidence=0.08386100386100385, lift=1.0)]),
 RelationRecord(items=frozenset({'47566'}), support=0.06586872586872587, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'47566'}), confidence=0.06586872586872587, lift=1.0)]),
 RelationRecord(items=frozenset({'85099B'}), support=0.08243243243243244, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'85099B'}), confidence=0.08243243243243244, lift=1.0)]),
 RelationRecord(items=frozenset({'85123A'}), support=0.08671814671814672, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'85123A'}), confidence=0.08671814671814672, lift=1.0)])]

In [ ]:
frequentItemsets[0]

RelationRecord(items=frozenset({'22423'}), support=0.08386100386100385, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22423'}), confidence=0.08386100386100385, lift=1.0)])

In [ ]:
frequentItemsets = list(apriori(transactions,min_support = 0.03))
frequentItemsets

[RelationRecord(items=frozenset({'20712'}), support=0.033667953667953665, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20712'}), confidence=0.033667953667953665, lift=1.0)]),
 RelationRecord(items=frozenset({'20719'}), support=0.03254826254826255, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20719'}), confidence=0.03254826254826255, lift=1.0)]),
 RelationRecord(items=frozenset({'20724'}), support=0.04054054054054054, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20724'}), confidence=0.04054054054054054, lift=1.0)]),
 RelationRecord(items=frozenset({'20725'}), support=0.062084942084942085, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'20725'}), confidence=0.062084942084942085, lift=1.0)]),
 RelationRecord(items=frozenset({'20726'}), support=0.04003861003861004, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozense

In [ ]:
len(frequentItemsets)

80

In [ ]:
frequentItemsets[-1] #itemset ตัวสุดท้าย #2-itemset #สินค้า '22697', '22699' ถูกซื้อเกิน 3% ของ transction ทั้งหมด

RelationRecord(items=frozenset({'22699', '22697'}), support=0.03027027027027027, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'22699', '22697'}), confidence=0.03027027027027027, lift=1.0), OrderedStatistic(items_base=frozenset({'22697'}), items_add=frozenset({'22699'}), confidence=0.7417218543046358, lift=17.1523178807947), OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22697'}), confidence=0.7, lift=17.1523178807947)])

support=0.03027027027027027: สินค้าคู่ '22697', '22699' ถูกซื้อ 3.027% ของ transactions ทั้งหมด

OrderedStatistic(items_base=frozenset({'22697'}), items_add=frozenset({'22699'}), confidence=0.7417218543046358, lift=17.1523178807947) แปลว่า 74% ของคนที่ซื้อ '22697' ก็จะซื้อ '22699' ด้วย

OrderedStatistic(items_base=frozenset({'22699'}), items_add=frozenset({'22697'}), confidence=0.7, lift=17.1523178807947)] แปลว่าในทางตรงกันข้าม 70% ของคนที่ซื้อ '22699' ก็จะซื้อ '22697' ด้วย

# HW 8 หา Pattern ที่น่าสนใจใน Data ตำรวจ

ต้องการดู Pattern ของ  "พื้นที่" ที่มีโอกาสต่อการเกิด "อาชญากรรมประเภทหนึ่ง" ก็มักจะมีโอกาสการเกิด "อาชญากรรมอีกประเภทหนึ่ง" ด้วย

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
path = '/content/drive/MyDrive/Dataset_dpdm25/2025-01'

In [47]:
import pandas as pd
import os

In [48]:
df = pd.read_csv(os.path.join(path,'2025-01-nottinghamshire-street.csv'),encoding='latin-1')

## ตรวจสอบข้อมูล

In [49]:
df

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
0,3d83fe51abba4d51d9b50c7c7252fd39f381526df5f6d8...,2025-01,Nottinghamshire Police,Nottinghamshire Police,-1.414628,53.044918,On or near Strelley Avenue,E01019462,Amber Valley 006E,Drugs,Local resolution,NaN
1,82058f096cb12c9a232f38bb04ddb565d7d4325f3c1e8f...,2025-01,Nottinghamshire Police,Nottinghamshire Police,-1.251084,53.142307,On or near Hardwick Avenue,E01027979,Ashfield 001A,Burglary,Investigation complete; no suspect identified,NaN
2,2f006e797f6ba92344b391cb73188494bbac22d7d5992a...,2025-01,Nottinghamshire Police,Nottinghamshire Police,-1.259196,53.141800,On or near St Andrews Crescent,E01027979,Ashfield 001A,Burglary,Investigation complete; no suspect identified,NaN
3,e8b280fd16e4ac986b3c14d414bf3ac830502e07271918...,2025-01,Nottinghamshire Police,Nottinghamshire Police,-1.253696,53.141676,On or near Supermarket,E01027979,Ashfield 001A,Drugs,Local resolution,NaN
4,358e2dd423f0fa63cfb24f03b12bb7e1be71f569423e8b...,2025-01,Nottinghamshire Police,Nottinghamshire Police,-1.253696,53.141676,On or near Supermarket,E01027979,Ashfield 001A,Other theft,Unable to prosecute suspect,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9543,231f16011fd10da9fe31f741408d2f150a1bd455c15297...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Under investigation,NaN
9544,24700e2d6d67b17a25b431593fd41f3cc178ee2f8ab8f1...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Unable to prosecute suspect,NaN
9545,0eda8615e5710293042e971b618c1c216a37129f92a260...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Action to be taken by another organisation,NaN
9546,be9c26093704a2ae6ad9c077f9ff9002a48bdcfc24aa4d...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Investigation complete; no suspect identified,NaN


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9548 entries, 0 to 9547
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Crime ID               7963 non-null   object 
 1   Month                  9548 non-null   object 
 2   Reported by            9548 non-null   object 
 3   Falls within           9548 non-null   object 
 4   Longitude              9475 non-null   float64
 5   Latitude               9475 non-null   float64
 6   Location               9548 non-null   object 
 7   LSOA code              9475 non-null   object 
 8   LSOA name              9475 non-null   object 
 9   Crime type             9548 non-null   object 
 10  Last outcome category  7963 non-null   object 
 11  Context                0 non-null      float64
dtypes: float64(3), object(9)
memory usage: 895.3+ KB


In [51]:
df.isnull().sum()

,0
Crime ID,1585
Month,0
Reported by,0
Falls within,0
Longitude,73
Latitude,73
Location,0
LSOA code,73
LSOA name,73
Crime type,0


### Drop na ใน LSOA Code

In [52]:
df[df['LSOA code'].isnull()]

,Crime ID,Month,Reported by,Falls within,Longitude,Latitude,Location,LSOA code,LSOA name,Crime type,Last outcome category,Context
9475,63a8d5e0a3bd772dc4aa71a487e391533481f46f172d43...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Burglary,Investigation complete; no suspect identified,NaN
9476,fdf5b4e091fef0599a5e2f4d13f4c8b988980c73077384...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Burglary,Awaiting court outcome,NaN
9477,41a428ef9d6cd8ca317b2ea502f7108ea35567965a1ed0...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Burglary,Investigation complete; no suspect identified,NaN
9478,49827b14fbd2224a8fefc9e7e852efc26f0920d736375f...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Burglary,Investigation complete; no suspect identified,NaN
9479,fe830dda8b3b114412cd3fa57ad76aeadca20e4764a910...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Burglary,Unable to prosecute suspect,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9543,231f16011fd10da9fe31f741408d2f150a1bd455c15297...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Under investigation,NaN
9544,24700e2d6d67b17a25b431593fd41f3cc178ee2f8ab8f1...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Unable to prosecute suspect,NaN
9545,0eda8615e5710293042e971b618c1c216a37129f92a260...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Action to be taken by another organisation,NaN
9546,be9c26093704a2ae6ad9c077f9ff9002a48bdcfc24aa4d...,2025-01,Nottinghamshire Police,Nottinghamshire Police,NaN,NaN,No Location,NaN,NaN,Violence and sexual offences,Investigation complete; no suspect identified,NaN


In [53]:
df.dropna(subset=['LSOA code'], inplace=True)

In [54]:
df.isnull().any()

,0
Crime ID,True
Month,False
Reported by,False
Falls within,False
Longitude,False
Latitude,False
Location,False
LSOA code,False
LSOA name,False
Crime type,False


In [55]:
df['Crime type'].unique()

array(['Drugs', 'Burglary', 'Other theft', 'Public order',
       'Violence and sexual offences', 'Other crime',
       'Anti-social behaviour', 'Criminal damage and arson',
       'Vehicle crime', 'Possession of weapons', 'Shoplifting',
       'Theft from the person', 'Bicycle theft', 'Robbery'], dtype=object)

**ประเภทของอาชญากรรม (Crime Type)**
<br> Anti-social behaviour	พฤติกรรมต่อต้านสังคม
<br> Bicycle theft	การลักขโมยจักรยาน
<br> Burglary	การย่องเบา, การบุกรุกเข้าไปลักทรัพย์
<br> Criminal damage and arson	การทำให้เสียทรัพย์และการลอบวางเพลิง
<br> Drugs	ยาเสพติด
<br> Other crime	อาชญากากรรมอื่นๆ
<br> Other theft	การลักขโมยอื่นๆ
<br> Possession of weapons	การครอบครองอาวุธ
<br> Public order	การก่อกวนความสงบเรียบร้อย
<br> Robbery	การชิงทรัพย์
<br> Shoplifting	การลักขโมยของในร้านค้า
<br> Theft from the person	การลักทรัพย์จากตัวบุคคล (เช่น ล้วงกระเป๋า)
<br> Vehicle crime	อาชญากรรมเกี่ยวกับยานพาหนะ
<br> Violence and sexual offences	ความรุนแรงและอาชญากรรมทางเพศ

## การเตรียมข้อมูลสำหรับ Apriori Algorithm

In [56]:
transactions = df.groupby('LSOA code')['Crime type'].apply(list)

In [57]:
transactions

,Crime type
LSOA code,
E01004339,[Violence and sexual offences]
E01005221,[Violence and sexual offences]
E01007610,[Drugs]
E01007753,[Criminal damage and arson]
E01010121,[Violence and sexual offences]
...,...
E01035507,"[Anti-social behaviour, Anti-social behaviour,..."
E01035508,"[Anti-social behaviour, Anti-social behaviour,..."
E01035509,[Public order]


## Association Rules

In [58]:
!pip install apyori

In [59]:
from apyori import apriori

In [60]:
frequentItemsets = list(apriori(transactions,min_support = 0.3))

In [61]:
frequentItemsets

[RelationRecord(items=frozenset({'Anti-social behaviour'}), support=0.6226666666666667, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Anti-social behaviour'}), confidence=0.6226666666666667, lift=1.0)]),
 RelationRecord(items=frozenset({'Burglary'}), support=0.3453333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Burglary'}), confidence=0.3453333333333333, lift=1.0)]),
 RelationRecord(items=frozenset({'Criminal damage and arson'}), support=0.5213333333333333, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Criminal damage and arson'}), confidence=0.5213333333333333, lift=1.0)]),
 RelationRecord(items=frozenset({'Other theft'}), support=0.4, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Other theft'}), confidence=0.4, lift=1.0)]),
 RelationRecord(items=frozenset({'Public order'}), support=0.37733333333333335, ordered_statistics=[Orde

In [62]:
frequentItemsets[-1]

RelationRecord(items=frozenset({'Criminal damage and arson', 'Anti-social behaviour', 'Violence and sexual offences'}), support=0.36533333333333334, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Criminal damage and arson', 'Anti-social behaviour', 'Violence and sexual offences'}), confidence=0.36533333333333334, lift=1.0), OrderedStatistic(items_base=frozenset({'Anti-social behaviour'}), items_add=frozenset({'Criminal damage and arson', 'Violence and sexual offences'}), confidence=0.5867237687366167, lift=1.2291699065711243), OrderedStatistic(items_base=frozenset({'Criminal damage and arson'}), items_add=frozenset({'Anti-social behaviour', 'Violence and sexual offences'}), confidence=0.70076726342711, lift=1.2366481119301942), OrderedStatistic(items_base=frozenset({'Violence and sexual offences'}), items_add=frozenset({'Criminal damage and arson', 'Anti-social behaviour'}), confidence=0.4287949921752739, lift=1.1051417324104997), OrderedStatistic(it

support=0.36533333333333334
<br> **36.5% ของพื้นที่ทั้งหมดในข้อมูลชุดนี้ มีอาชญากรรม ทั้ง 3 ประเภทนี้ ได้แก่ Violence and sexual offences, Anti-social behaviour, Criminal damage and arson เกิดขึ้นครบทั้งหมด**



---



OrderedStatistic(items_base=frozenset({'Violence and sexual offences'}), items_add=frozenset({'Anti-social behaviour', 'Criminal damage and arson'}), confidence=0.4287949921752739, lift=1.1051417324104997)
<br>**42.87% ในพื้นที่ที่เกิดคดี 'ความรุนแรงและอาชญากรรมทางเพศ' มีโอกาสที่จะเกิดคดี  'พฤติกรรมต่อต้านสังคม' และ 'การทำให้เสียทรัพย์และการลอบวางเพลิง' ร่วมด้วย**

OrderedStatistic(items_base=frozenset({'Anti-social behaviour', 'Criminal damage and arson'}), items_add=frozenset({'Violence and sexual offences'}), confidence=0.9415807560137457, lift=1.1051417324104997)
<br> **94.16% ในพื้นที่ที่เกิดทั้งคดี 'พฤติกรรมต่อต้านสังคม' และ 'การทำให้เสียทรัพย์และการลอบวางเพลิง' มีโอกาสที่จะเกิดคดี 'ความรุนแรงและอาชญากรรมทางเพศ' ร่วมด้วย**



---



OrderedStatistic(items_base=frozenset({'Anti-social behaviour'}), items_add=frozenset({'Violence and sexual offences', 'Criminal damage and arson'}), confidence=0.5867237687366167, lift=1.2291699065711243)
<br>**58.67% ในพื้นที่ที่เกิดคดี 'พฤติกรรมต่อต้านสังคม' มีโอกาสที่จะเกิดคดี  'ความรุนแรงและอาชญากรรมทางเพศ' และ 'การทำให้เสียทรัพย์และการลอบวางเพลิง'  ร่วมด้วย**

OrderedStatistic(items_base=frozenset({'Violence and sexual offences', 'Criminal damage and arson'}), items_add=frozenset({'Anti-social behaviour'}), confidence=0.7653631284916201, lift=1.2291699065711243)
<br>**76.54% ในพื้นที่ที่เกิดทั้งคดี 'ความรุนแรงและอาชญากรรมทางเพศ' และ'การทำให้เสียทรัพย์และการลอบวางเพลิง'  มีโอกาสที่จะเกิดคดี ''พฤติกรรมต่อต้านสังคม' ร่วมด้วย**



---



OrderedStatistic(items_base=frozenset({'Criminal damage and arson'}), items_add=frozenset({'Violence and sexual offences', 'Anti-social behaviour'}), confidence=0.70076726342711, lift=1.2366481119301942)
<br>**70.08% ในพื้นที่ที่เกิดคดี 'การทำให้เสียทรัพย์และการลอบวางเพลิง' มีโอกาสที่จะเกิดคดี  'ความรุนแรงและอาชญากรรมทางเพศ' และ 'พฤติกรรมต่อต้านสังคม' ร่วมด้วย**

OrderedStatistic(items_base=frozenset({'Violence and sexual offences', 'Anti-social behaviour'}), items_add=frozenset({'Criminal damage and arson'}), confidence=0.6447058823529412, lift=1.2366481119301942)
<br>**64.47% ในพื้นที่ที่เกิดทั้งคดี 'ความรุนแรงและอาชญากรรมทางเพศ' และ'พฤติกรรมต่อต้านสังคม'  มีโอกาสที่จะเกิดคดี 'การทำให้เสียทรัพย์และการลอบวางเพลิง' ร่วมด้วย**



---





>  ค่า Lift ที่มากกว่า 1 แสดงว่ามีความสัมพันธ์กันจริง (ไม่ใช่เรื่องบังเอิญ)


> ถ้าจัดทีมตำรวจ ไปสืบ แล้วเจอว่า 'พฤติกรรมต่อต้านสังคม' และ 'การทำให้เสียทรัพย์และการลอบวางเพลิง' ก็ส่งตำรวจที่เชี่ยวชาญไปเพิ่มเพราะโอกาสสูงที่จะเกิด 'ความรุนแรงและอาชญากรรมทางเพศ' ร่วมด้วย





---



# ข้อมูลแต่ละอัน ที่เป็น Text สัมพันธ์กันยังไงในแต่ละ Column

ข้อมูลแต่ละอัน ที่เป็น Text สัมพันธ์กันยังไงในแต่ละ Column

In [64]:
path = '/content/drive/MyDrive/Dataset_dpdm25/2025-01'
df_nth = pd.read_csv(os.path.join(path,'2025-01-nottinghamshire-stop-and-search.csv'),encoding='latin-1')

In [65]:
df_nth.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing
0,Person search,2025-01-01T00:39:00+00:00,False,NaN,54.007280,-2.896877,Male,10-17,Other ethnic group - Not stated,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False
1,Person search,2025-01-01T02:00:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,True,False
2,Person search,2025-01-01T02:08:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
3,Person search,2025-01-01T02:13:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Asian/Asian British - Any other Asian background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False
4,Person search,2025-01-01T02:25:00+00:00,False,NaN,54.007280,-2.896877,Male,over 34,Other ethnic group - Not stated,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False


In [66]:
transactions_nth = df_nth[['Gender', 'Age range', 'Self-defined ethnicity', 'Officer-defined ethnicity', 'Object of search']].apply(lambda x: x.tolist(), axis=1).tolist()

In [69]:
transactions_nth[:1]

[['Male',
  '10-17',
  'Other ethnic group - Not stated',
  'Black',
  'Offensive weapons']]

In [70]:
df_nth['LOI'] = transactions_nth

In [71]:
df_nth.head()

,Type,Date,Part of a policing operation,Policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,LOI
0,Person search,2025-01-01T00:39:00+00:00,False,NaN,54.007280,-2.896877,Male,10-17,Other ethnic group - Not stated,Black,Police and Criminal Evidence Act 1984 (section 1),Offensive weapons,A no further action disposal,False,False,"[Male, 10-17, Other ethnic group - Not stated,..."
1,Person search,2025-01-01T02:00:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,Arrest,True,False,"[Male, 10-17, Other ethnic group - Not stated,..."
2,Person search,2025-01-01T02:08:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Other ethnic group - Not stated,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, 10-17, Other ethnic group - Not stated,..."
3,Person search,2025-01-01T02:13:00+00:00,False,NaN,53.969736,-2.865897,Male,10-17,Asian/Asian British - Any other Asian background,Other,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, 10-17, Asian/Asian British - Any other ..."
4,Person search,2025-01-01T02:25:00+00:00,False,NaN,54.007280,-2.896877,Male,over 34,Other ethnic group - Not stated,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,False,False,"[Male, over 34, Other ethnic group - Not state..."


In [84]:
df_nth_transactions = df_nth[['Date','LOI']]
df_nth_transactions[:2]

,Date,LOI
0,2025-01-01T00:39:00+00:00,"[Male, 10-17, Other ethnic group - Not stated,..."
1,2025-01-01T02:00:00+00:00,"[Male, 10-17, Other ethnic group - Not stated,..."


In [85]:
df_nth_transactions['LOI'][0]

['Male',
 '10-17',
 'Other ethnic group - Not stated',
 'Black',
 'Offensive weapons']

In [86]:
df_nth_transactions.groupby('Date')['LOI'].apply(list)

,LOI
Date,
2025-01-01T00:39:00+00:00,"[[Male, 10-17, Other ethnic group - Not stated..."
2025-01-01T02:00:00+00:00,"[[Male, 10-17, Other ethnic group - Not stated..."
2025-01-01T02:08:00+00:00,"[[Male, 10-17, Other ethnic group - Not stated..."
2025-01-01T02:13:00+00:00,"[[Male, 10-17, Asian/Asian British - Any other..."
2025-01-01T02:25:00+00:00,"[[Male, over 34, Other ethnic group - Not stat..."
...,...
2025-01-31T16:35:00+00:00,"[[Male, 18-24, White - English/Welsh/Scottish/..."
2025-01-31T16:48:00+00:00,"[[Female, 25-34, Other ethnic group - Not stat..."
2025-01-31T20:43:00+00:00,"[[Male, 18-24, Black/African/Caribbean/Black B..."


In [ ]:
#ใช้ transactions_nth เลย เพราะจัดรูปมาแล้ว

In [89]:
list_apri = list(apriori(transactions_nth, min_support = 0.05))

In [98]:
list_apri[:5]

[RelationRecord(items=frozenset({'10-17'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'10-17'}), confidence=0.20121951219512196, lift=1.0)]),
 RelationRecord(items=frozenset({'18-24'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24'}), confidence=0.20121951219512196, lift=1.0)]),
 RelationRecord(items=frozenset({'25-34'}), support=0.18597560975609756, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'25-34'}), confidence=0.18597560975609756, lift=1.0)]),
 RelationRecord(items=frozenset({'Article for use in theft'}), support=0.8567073170731707, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Article for use in theft'}), confidence=0.8567073170731707, lift=1.0)]),
 RelationRecord(items=frozenset({'Asian'}), support=0.08231707317073171, ordered_statistics=[OrderedStatistic(items_base

# หาว่าตำรวจ Black มีความสัมพันธ์กับอะไรบ้าง

In [100]:
police_black = list()
for result in list_apri:
  if 'Black' in result[0]: #str(result[0])
    police_black.append(result)

In [105]:
len(police_black)

12

In [104]:
police_black

[RelationRecord(items=frozenset({'Black'}), support=0.17682926829268292, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black'}), confidence=0.17682926829268292, lift=1.0)]),
 RelationRecord(items=frozenset({'Black', '10-17'}), support=0.051829268292682924, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black', '10-17'}), confidence=0.051829268292682924, lift=1.0), OrderedStatistic(items_base=frozenset({'10-17'}), items_add=frozenset({'Black'}), confidence=0.25757575757575757, lift=1.4566353187042842), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'10-17'}), confidence=0.29310344827586204, lift=1.456635318704284)]),
 RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.057926829268292686, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), 

In [107]:
for i,rule in enumerate(police_black):
  print(i,rule)

0 RelationRecord(items=frozenset({'Black'}), support=0.17682926829268292, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black'}), confidence=0.17682926829268292, lift=1.0)])
1 RelationRecord(items=frozenset({'Black', '10-17'}), support=0.051829268292682924, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black', '10-17'}), confidence=0.051829268292682924, lift=1.0), OrderedStatistic(items_base=frozenset({'10-17'}), items_add=frozenset({'Black'}), confidence=0.25757575757575757, lift=1.4566353187042842), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'10-17'}), confidence=0.29310344827586204, lift=1.456635318704284)])
2 RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.057926829268292686, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}),

<br> ใน Transaction ทั้้งหมด มี ตำรวจ Black ชอบจับ Article for use in theft 14.94%
<br> มี ตำรวจ Black ชอบจับคนที่เป็นผู้ชาย 17%
<br> **มี ตำรวจ Black ชอบจับคนที่เป็นผู้ชาย และมี Article for use in theft 14.3%**
<br> Article for use in theft อุปกรณ์ที่ใช้ในการโจรกรรม

In [109]:
police_black[8]

RelationRecord(items=frozenset({'Black', 'Article for use in theft', 'Male'}), support=0.14329268292682926, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black', 'Article for use in theft', 'Male'}), confidence=0.14329268292682926, lift=1.0), OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'Black', 'Male'}), confidence=0.16725978647686832, lift=0.9796644636502286), OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'Article for use in theft', 'Male'}), confidence=0.8103448275862069, lift=1.0848698099929626), OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Black', 'Article for use in theft'}), confidence=0.16607773851590105, lift=1.1117040455758274), OrderedStatistic(items_base=frozenset({'Black', 'Article for use in theft'}), items_add=frozenset({'Male'}), confidence=0.9591836734693877, lift=1.1117040455758274), OrderedStatistic(items_base=frozenset({'Article for us

In [111]:
for v in police_black[8][2]:
  print(v)

OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Black', 'Article for use in theft', 'Male'}), confidence=0.14329268292682926, lift=1.0)
OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'Black', 'Male'}), confidence=0.16725978647686832, lift=0.9796644636502286)
OrderedStatistic(items_base=frozenset({'Black'}), items_add=frozenset({'Article for use in theft', 'Male'}), confidence=0.8103448275862069, lift=1.0848698099929626)
OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'Black', 'Article for use in theft'}), confidence=0.16607773851590105, lift=1.1117040455758274)
OrderedStatistic(items_base=frozenset({'Black', 'Article for use in theft'}), items_add=frozenset({'Male'}), confidence=0.9591836734693877, lift=1.1117040455758274)
OrderedStatistic(items_base=frozenset({'Article for use in theft', 'Male'}), items_add=frozenset({'Black'}), confidence=0.19183673469387755, lift=1.0848698099929628)
OrderedStatistic(item

(base = black)
81% ของตำรวจ Black  มักจะเรียกค้นผู้ชายแล้วจะเจอว่า มีอุปกรณ์ที่ใช้ในการโจรกรรม

#หาว่า ช่วงอายุ วัยรุ่น 18-24 มีความสัมพันธ์กับอะไรบ้าง

In [112]:
teen = list()
for result in list_apri:
  if '18-24' in result[0]: #str(result[0])
    teen.append(result)

In [114]:
len(teen)

20

In [115]:
for i,rule in enumerate(teen):
  print(i,rule)

0 RelationRecord(items=frozenset({'18-24'}), support=0.20121951219512196, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24'}), confidence=0.20121951219512196, lift=1.0)])
1 RelationRecord(items=frozenset({'18-24', 'Article for use in theft'}), support=0.1676829268292683, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Article for use in theft'}), confidence=0.1676829268292683, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), items_add=frozenset({'Article for use in theft'}), confidence=0.8333333333333334, lift=0.9727164887307237), OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'18-24'}), confidence=0.19572953736654805, lift=0.9727164887307236)])
2 RelationRecord(items=frozenset({'18-24', 'Black'}), support=0.057926829268292686, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Black'}), confidence=0.05792682

In [119]:
teen[1]

RelationRecord(items=frozenset({'18-24', 'Article for use in theft'}), support=0.1676829268292683, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Article for use in theft'}), confidence=0.1676829268292683, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), items_add=frozenset({'Article for use in theft'}), confidence=0.8333333333333334, lift=0.9727164887307237), OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'18-24'}), confidence=0.19572953736654805, lift=0.9727164887307236)])

<br> 16% ของผู้ต้องหาที่ถูกจับทั้งหมด อายุ 18-24 และ  มีอุปกรณ์ในการโจรกรรม
<br> 83% ของผู้ต้องหาอายุ 18-24 จะมีอุปกรณ์ในการโจรกรรม
<br> **แต่ ค่า lift<1**

In [120]:
teen[3]

RelationRecord(items=frozenset({'18-24', 'Male'}), support=0.18292682926829268, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Male'}), confidence=0.18292682926829268, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), items_add=frozenset({'Male'}), confidence=0.9090909090909091, lift=1.053646000642467), OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'18-24'}), confidence=0.21201413427561838, lift=1.053646000642467)])

In [121]:
teen[7]

RelationRecord(items=frozenset({'18-24', 'Article for use in theft', 'Male'}), support=0.1524390243902439, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'18-24', 'Article for use in theft', 'Male'}), confidence=0.1524390243902439, lift=1.0), OrderedStatistic(items_base=frozenset({'18-24'}), items_add=frozenset({'Article for use in theft', 'Male'}), confidence=0.7575757575757575, lift=1.0142238713667284), OrderedStatistic(items_base=frozenset({'Article for use in theft'}), items_add=frozenset({'18-24', 'Male'}), confidence=0.17793594306049823, lift=0.9727164887307237), OrderedStatistic(items_base=frozenset({'Male'}), items_add=frozenset({'18-24', 'Article for use in theft'}), confidence=0.17667844522968199, lift=1.053646000642467), OrderedStatistic(items_base=frozenset({'18-24', 'Article for use in theft'}), items_add=frozenset({'Male'}), confidence=0.9090909090909091, lift=1.053646000642467), OrderedStatistic(items_base=frozenset({'18-24', 'Male'}), 

อยากจะรู้ว่าอาชญากรรมที่เกิดขึ้นใน 1 วัน มักจะมีคสพ อะไรกันบ้าง ต้อง groupby "Date"